In [86]:
import pandas as pd
import ast

In [87]:
df_stops_total = pd.read_csv("../../../../1_data_fahrplanbuch/final-tables/stations.csv")
df_1963 = pd.read_csv('../initial/stops_df_1963-initial.csv')

In [88]:
df_1963.head(3)

,Unnamed: 0,station_id,stop_name,line_count,type,in_lines
0,0,1,Pankow (Vinetastr.),1,u-bahn,['A']
1,1,2,Schönhauser Allee,2,u-bahn,['A']
2,2,2,Schönhauser Allee,2,strassenbahn,['3']


In [89]:
df_1963.drop(columns=['Unnamed: 0'], inplace=True)
df_1963["location"] = ""
df_1963["identifier"] = ""
df_1963["stop_id_other"] = ""
df_1963["previous_in_lines"] = ""

In [90]:
df_stops_total.tail(3)

,stop_id,stop_name,type,location,in_lines,identifier,stop_description
19728,19892665,neuhofer Strasse,bus,"52.411694901653036, 13.503489970696608",{'58'},NaN,NaN
19729,19892666,Öschebronner Weg,bus,"52.61154475375042, 13.337669105320359",{'20'},NaN,NaN
19730,19892667,Übergang,bus,"52.543078667825824, 13.157884773521127",{'63'},NaN,NaN


In [91]:
# Function to convert string representation of set to list
def convert_to_list(value):
    try:
        # Use ast.literal_eval to safely convert string to set
        items = ast.literal_eval(value)
        
        # Check if the result is a set, and convert it to a list
        if isinstance(items, set):
            return list(items)
        # Check if the result is an integer, and convert it to a list with a single element
        elif isinstance(items, int):
            return [items]
        # For other types, return a list with the original value
        else:
            return [value]
    except (SyntaxError, ValueError):
        # Handle the case where literal_eval fails (e.g., invalid string)
        # Try to convert the value to an integer, and return a list with a single element if successful
        try:
            return [int(value)]
        except ValueError:
            return [value]

# Apply the function to the 'in_lines' column
df_stops_total['in_lines'] = df_stops_total['in_lines'].apply(convert_to_list)
df_1963['in_lines'] = df_1963['in_lines'].apply(convert_to_list)

In [92]:
df_stops_total['in_lines'] = df_stops_total['in_lines'].apply(lambda x: ','.join(map(str, x)))
df_1963['in_lines'] = df_1963['in_lines'].apply(lambda x: ','.join(map(str, x)))

In [93]:
df_stops_total.head(3)

,stop_id,stop_name,type,location,in_lines,identifier,stop_description
0,19460,Adlershof,s-bahn,"52.434722222222,13.541388888889","['KBS 100a', 'KBS 100c']",Q323551,NaN
1,19461,Akazienallee,bus,"52.47849809201335, 13.607179083668733",['D'],NaN,NaN
2,19462,Alexanderplatz,bus,"52.52005246558073, 13.413957243449447","['1', '9']",NaN,NaN


I have two dataframes df_long and df_short, I want to add the information from df_long columns combined_location and combined_identifier to df_short if there is a match between the original_name column from df_long with the stop_name column of df_short. I have to create combined_location and combined_identifier columns in df_short first

In [94]:
df_stops_total['type'] = df_stops_total['type'].replace('bus', 'Autobus')

In [96]:
for index, row in df_1963.iterrows():
    # Check if row["stop_name"] is in df_1960["original_name"]
    if row["stop_name"] in df_stops_total["stop_name"].values:
        # Find matching rows in df_1960
        matching_rows = df_stops_total[df_stops_total["stop_name"] == row["stop_name"]]
        
        # Check if those two rows have matching row["type"] values
        matching_rows = matching_rows[matching_rows["type"] == row["type"]]
        
        # Check if there are any matching rows
        if not matching_rows.empty:
            df_1963.at[index, 'location'] = '+ '.join(matching_rows['location'].astype(str).values)
            df_1963.at[index, 'identifier'] = '+ '.join(matching_rows['identifier'].astype(str).values)
            df_1963.at[index, 'stop_description'] = '+ '.join(matching_rows['stop_description'].astype(str).values)
            df_1963.at[index, 'stop_id_other'] = '+ '.join(matching_rows['stop_id'].astype(str).values)
            df_1963.at[index, 'previous_in_lines'] = '+ '.join(matching_rows['in_lines'].astype(str).values)
        else:
            # Handle the case where there are no matching rows in df_1960
            df_1963.at[index, 'location'] = ''
            df_1963.at[index, 'identifier'] = ''
            df_1963.at[index, 'stop_description'] = ''
            df_1963.at[index, 'stop_id_other'] = ''
            df_1963.at[index, 'previous_in_lines'] = ''




        # # Convert values to strings and replace NaN with empty strings
        # combined_location = " + ".join(matching_rows["combined_location"].astype(str).fillna(''))
        # combined_identifier = " + ".join(matching_rows["combined_identifier"].astype(str).fillna(''))
            
        # # Update row["location"] and row["identifier"]
        # row["location"] += " + " + combined_location
        # row["identifier"] += " + " + combined_identifier

# Now, df_1961 should have updated "location" and "identifier" columns based on the conditions.


In [97]:
# Function to split string values containing '+' and convert to a unique string
def split_values(cell_value):
    unique_values = sorted(set(element.strip() for element in cell_value.split('+')))
    return '+'.join(unique_values)

# Apply the function to each cell in the specified columns
df_1963['location'] = df_1963['location'].apply(split_values)
df_1963['identifier'] = df_1963['identifier'].apply(split_values)

In [98]:
df_1963.drop(columns=['stop_description', "stop_id_other"], inplace=True)

In [99]:
# Function to concatenate unique values in other columns
def concat_unique_values(series):
    unique_values = set(series)
    return '+ '.join(unique_values)

# Group by specified columns and aggregate the other columns
df_1963 = df_1963.groupby(['stop_name', 'type', 'in_lines', 'location']).agg({
    'identifier': concat_unique_values,
    'previous_in_lines': concat_unique_values
}).reset_index()

In [100]:
df_1963.to_csv('1963-stops-combined.csv')